# Examples of Richardson Extrapolation

In [ ]:
using Trapz
using Plots

## Differentiation Example
Find the derivative of $f(x) = \ln x$ at $x = 2$.  Recall that the forward difference method is $\mathrm{O}(h^1)$,
$$
f'(x_0) = \underbrace{\frac{f(x_0 + h)-f(x_0)}{h}}_{N_1(h)} + \mathrm{O}(h^1)
$$

In [ ]:
x = 2;
h = 0.01;
estimate1 = (log(x+h) - log(x))/h;
estimate2 = (log(x+h/2) - log(x))/(h/2);
richardson_est = 2*estimate2 - estimate1;
println("Exact = $(0.5)")
println("First estimate = $(estimate1), error = $(abs(estimate1 - 0.5))")
println("Second estimate = $(estimate2), error = $(abs(estimate2 - 0.5))")
println("Richardson estimate = $(richardson_est), error = $(abs(richardson_est - 0.5))")


## Integration Example
Compute
$$
\int_1^2 x \ln x dx
$$
using trapezoidal rule and Richardson extrapolation.  Recall that composite trapezoidal rule has a $\mathrm{O}(h^2)$ error.

In [ ]:
f = x -> x * log(x);
h = 0.1;
x1 = 1:h:2;
x2 = 1:h/2:2;

estimate1 = trapz(x1, f.(x1));
estimate2 = trapz(x2, f.(x2));
richardson = 4/3 * estimate2 - 1/3 * estimate1;

exact = -0.75 + log(4);

println("Exact = $(exact)")
println("First estimate = $(estimate1), error = $(abs(estimate1 - exact))")
println("Second estimate = $(estimate2), error = $(abs(estimate2 - exact))")
println("Richardson estimate = $(richardson), error = $(abs(richardson - exact))")
println("Richardson estimate/16 = $(richardson/16), error = $(abs(richardson - exact)/16)")

Composite trapezoidal, empirically, expands in powers of $h^2$:
$$
M =  N_1(h) + K_2 h^2 + K_4 h^4 + K_6 h^6 + \ldots
$$

# Sqrt Revisited
Recall that if we use composite trapezoidal rule for a function like $\sqrt{x}$, problems can arise:
$$
\frac{2}{3}=\int_0^1 \sqrt{x} = \text{Composite Trapezoidal Rule} + \mathrm{O}(h^{3/2})
$$
We can use Richardson extrapolation here.  Denoting  $N_1(h)$ the composite trapezoidal rule method,
$$
M = N_1(h) + K h^{3/2} + \text{higher order terms},
$$
we can derive
$$
N_2(h) = \frac{2^{3/2} N_1(h/2) - N_1(h)}{2^{3/2}-1}
$$

In [ ]:
function trapz_richardson(f, a, b, h)
    x1 = a:h:b
    x2 = a:h/2:b

    estimate1 = trapz(x1, f.(x1))
    estimate2 = trapz(x2, f.(x2))
    richardson = (2^(3/2) * estimate2 - estimate1) / (2^(3/2) - 1);

    return richardson
end

In [ ]:
f =x-> sqrt(x);
a = 0;
b = 1;

n_vals = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048];
err_vals = Float64[];
∫f_exact = 2/3;

for n in n_vals
    h = 1/n;
    ∫f_richard = trapz_richardson(f, a, b, h)
    push!(err_vals, abs(∫f_exact - ∫f_richard));
    println("n = $n: Richardson Estimate = $∫f_richard, Absolute Error = $(abs(∫f_exact - ∫f_richard))");
end

In [ ]:
h_vals   = 2 ./(n_vals);    
scatter(h_vals, err_vals, label= "Data", xscale=:log10, yscale=:log10, xlabel="h", ylabel="Absolute Error", legend=:bottomright)
plot!(h_vals,.001 * h_vals, label="∝h", ls=:dash)
plot!(h_vals, .001 * h_vals.^2, label="∝h²", ls=:dash)

Now we have what appears to be an $\mathrm{O}(h^2)$ method.